<div class="markdown-google-sans">

# Universidade de Brasília
## Instituto de Física
---
### Métodos Computacionais A (MCA) - Prof. Bernhard Enders
---

</div>

<div class="markdown-google-sans">

## **➲ Aula 01 - Introdução à Disciplina**

### ➥ O que será visto?

* Funções especiais. 
* Zero de funções.
* Integração aproximação de funções.
* Álgebra linear, autovalores e autovetores.
* Equações não lineares.
* Equações diferenciais ordinárias.
* Outros tópicos em métodos numéricos aplicados à Física...

### ➥ Como será visto?

Utilizaremos a linguagem de programação Python para implementar as soluções numéricas dos problemas propostos.

### ➥ Por que usar Python?

Python é uma linguagem simples e fácil de aprender. Além disso, Python é uma linguagem utilizada em muitas áreas (script, desktop, web, mobile, IA, Ciência de Dados etc...). Já Fortran é uma linguagem bem restrita, utilizada majoritariamente em aplicações HPC devido a seu alto desempenho.

### ➥ Não está convencido? 

Vamos apresentar então alguns exemplos/vantagens de utilizar Python na tentativa de lhe convencer de que Python é uma escolha melhor e mais moderna para a nossa disciplina.









</div>

In [ ]:
def poly(*args, xvals=None):
    """
    Função que retorna uma lista contendo as ordenadas de um polinômio 
    de grau arbitrário.
    """
    if xvals is None:
        xvals = np.arange(0, 10, 0.01)
    ret = []
    degree = int(len(args) - 1)
    for x in xvals:
        y = 0.0
        for i in range(degree + 1):
            y += float(args[i]) * x**(degree - i)
        ret.append(y)
    return ret


def print_poly(*args):
    '''Prints the polynomial in a readable format'''
    y = "f(x) = "
    degree = len(args) - 1
    for i in range(len(args)):
        y += str(args[i]) if int(args[i]) != 1 else ""
        y += "x^"+str(degree - i) if int(degree - i) != 0 else " "
    return y.replace("+", " + ").replace("-", " - ")

In [ ]:
"""
Plota um polinômio a fim de encontrar suas raízes.
"""

import numpy as np
import plotly.express as px

# define os valores de x e y que serão utilizados
x = np.arange(-3.5, 3.5, 0.01)
y = poly(1, -2, -10, 20, 9, -14, xvals=x)

# plot interativo
fig = px.line(x=x, y=y, title=f'Função Polinomial')
fig.show()

<div class="markdown-google-sans">

Com apenas um pouco mais de linhas de código podemos construir uma **aplicação web** sem a necessidade de usar explicitamente HTML, CSS, Javascript etc... tampouco configurar um servidor web, uma base de dados etc... 

</div>

```python
import numpy as np
import plotly.express as px
import streamlit as st


# page title and header
title = "Encontrando as raízes de uma função polinomial"
st.set_page_config(
    page_title=title,
    page_icon=":1234:",
    layout="wide")
st.title(title)

# define our polynomial via text_input
st.write('### Informe abaixo os coeficientes do polinômio (separados por vírgula):')
coefs_str = st.text_input('Coeficientes', '+1, -2, -10, +20, +9, -14')
coefs = coefs_str.replace(" ", "").split(",")

# print polynomial function
st.write('O polinômio informado foi: ')
st.latex(print_poly(*coefs))

# x & y values based on slider from user input
start_x, end_x = st.select_slider(
    'Selecione um intervalo para achar as raízes:',
    options=[*range(-10, 11)],
    value=(-4, 5))
x = np.linspace(start_x, end_x, 5000)
y = poly(*coefs, xvals=x)

# plot interativo
fig = px.line(x=x, y=y, title=f'Plot da Função Polinomial')
st.plotly_chart(fig, use_container_width=True)
```

<div class="markdown-google-sans">

### ➥ Mas Python é muito lenta!!!

Será? Qual são as suas necessidades de desempenho? Vejamos um comparativo de **Python vs Fortran** a seguir.

#### Autovalores e autovetores da uma matrix complexa

Os programas a seguir calculam todos os autovalores e todos os autovetores de uma matriz hermitiana. 

```python
# python
import time
import numpy as np
from scipy.linalg import eigh

# record start time
start = time.time()

# square matrix dimension
n = 4096

# generate our random (real and imaginary) matrix elements
print('01. Generating random numbers for hermitian matrix... ')
A = np.random.rand(n,n)
B = np.random.rand(n,n)

print('02. Building our positive definite hermitian matrix...')
# build complex matrix from real ones
C = A + 1j*B

# ensure its hermitian
C = C * C.conj().T

# positive-definite
np.fill_diagonal(C, C.diagonal() + n)

# compute eigenvalues and eigenvectors
print('03. Computing all eigenvalues+eigenvectors using numpy eigh...')
w, v = eigh(C)

# save end time
end = time.time()
print(f"Numpy eigh took: {end-start:8.3f} seconds")
```



```fortran
! ifort -O2 -i8 -mcmodel=medium -qmkl=parallel ./hermitianEigen-small.f90 -o ./hermitianEigen-small \ 
! ${MKLROOT}/lib/intel64/libmkl_lapack95_ilp64.a -L${MKLROOT}/lib/intel64 -lpthread -lm -ldl


program hermitianEigen

    use, intrinsic :: iso_fortran_env
    use lapack95, only: heevd

    ! default integer precision
    integer, parameter :: ip = INT64
    ! default real precision
    integer, parameter :: fp = REAL64
    ! square matrix dimension
    integer, parameter :: n = 4096
    ! real matrices
    real(fp), dimension(:), allocatable   :: w
    real(fp), dimension(:,:), allocatable :: a
    ! complex matrix
    complex(fp), dimension(:,:), allocatable :: c, tmp
    integer(ip) :: start, end, rate
    intrinsic :: random_seed, random_number

    ! record start time
    call system_clock (count_rate=rate)
    call system_clock (count=start)

    ! allocate memory
    allocate(w(n), a(n,n), c(n,n), tmp(n,n))

    ! generate our random (real and imaginary) matrix elements
    print*, '01. Generating random numbers for hermitian matrix...'
    print*, '02. Building our positive definite hermitian matrix...'
    call random_number(a)
    tmp%re = a
    call random_number(a)
    tmp%im = a

    ! free some memory
    deallocate(a)

    ! ensure its hermitian
    !c = c * conjg(transpose(c)) ! FATAL: produces seg fault with ifort, requires tmp to avoid!
    c = tmp * conjg(transpose(tmp))

    deallocate(tmp)

    ! positive-definite
    do concurrent (i=1:n)
        c(i,i) = c(i,i) + n
    end do

    call heevd(c, w, 'V')

    call system_clock (count=end)
    write(*,"(a,f8.3,a)") "ZHEEVD took: ", real(end - start)/real(rate), " seconds"

end program hermitianEigen
```
</div>